In [1]:
!wget https://github.com/DanteNoguez/gpt-wilde/raw/main/wilde.txt

--2023-01-17 22:55:06--  https://github.com/DanteNoguez/gpt-wilde/raw/main/wilde.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DanteNoguez/gpt-wilde/main/wilde.txt [following]
--2023-01-17 22:55:07--  https://raw.githubusercontent.com/DanteNoguez/gpt-wilde/main/wilde.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4818475 (4.6M) [text/plain]
Saving to: ‘wilde.txt’

wilde.txt           100%[===================>]   4.59M  --.-KB/s    in 0.03s   

2023-01-17 22:55:07 (173 MB/s) - ‘wilde.txt’ saved [4818475/4818475]



In [2]:
with open('wilde.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
len(text)

4763434

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !&'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]abcdefghijklmnopqrstuvwxyz£ÀÆÈÉàáâäæçèéêëîïóôöùûüÿŒœ–—‘’“”
112


In [15]:
#character level tokenizer
stoi = {ch:i for i,ch in enumerate(chars)} #lookup table, mapping from characters to integers
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder: take a string, output integers
decode = lambda l: ''.join([itos[i] for i in l]) #decoder: take integers, output string

In [16]:
encode("hello there"), decode(encode("hello there"))

([61, 58, 65, 65, 68, 1, 73, 61, 58, 71, 58], 'hello there')

In [18]:
import torch

#encoding dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # todo el texto encoded

torch.Size([4763434]) torch.int64
tensor([  0,  43,  56,  58,  67,  58,  10,   1, 107,   1,  25,   1,  42,  74,
         72,  72,  62,  54,  67,   1,  33,  67,  67,  10,   0,   0,   0,   0,
         36,  54,  71,  60,  58,   1,  57,  68,  68,  71,   1,  68,  69,  58,
         67,  62,  67,  60,   1,  68,  67,   1,  72,  67,  68,  76,  78,   1,
         65,  54,  67,  57,  72,  56,  54,  69,  58,   1,  54,  73,   1,  55,
         54,  56,  64,   1,  68,  59,   1,  72,  73,  54,  60,  58,  10,   0,
          0,   0,   0,  40,  58,  73,  58,  71,   1,  43,  54,  55,  68,  74,
         71,  68])


In [19]:
#split data into train and validation sets
n = int(0.9*len(data)) #90% train, 10%val
train_data = data[:n]
val_data = data[n:]

In [30]:
import numpy as np
block_size = 8 #or context length
train_data[:block_size+1], decode(train_data[:block_size+1].numpy())

(tensor([  0,  43,  56,  58,  67,  58,  10,   1, 107]), '\nScene. —')

In [31]:
#example
x = train_data[:block_size] #context
y = train_data[1:block_size+1] #prediction
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context}, the target is {target}")

when input is tensor([0]), the target is 43
when input is tensor([ 0, 43]), the target is 56
when input is tensor([ 0, 43, 56]), the target is 58
when input is tensor([ 0, 43, 56, 58]), the target is 67
when input is tensor([ 0, 43, 56, 58, 67]), the target is 58
when input is tensor([ 0, 43, 56, 58, 67, 58]), the target is 10
when input is tensor([ 0, 43, 56, 58, 67, 58, 10]), the target is 1
when input is tensor([ 0, 43, 56, 58, 67, 58, 10,  1]), the target is 107
